In [1]:
import os
import timeit
import pyarrow
import pandas as pd
import numpy as np
import time
from itertools import compress  

from sklearn.tree import DecisionTreeClassifier
from six import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.tree import export_graphviz
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [2]:
print(os.getcwd())

/Users/marcin/Documents/VIMSS/ontology/Sample/biosample-analysis/src/notebooks


Below we read the data from a compressed parquet file into a pandas df

In [3]:
start_time = time.time()

df = pd.read_parquet("../../harmonized-table.parquet.gzip")

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

seconds 170.05871677398682


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14300584 entries, 0 to 14300583
Columns: 464 entries, accession to trophic_level
dtypes: object(464)
memory usage: 49.4+ GB


In [5]:
start_time = time.time()

df_describe = df.describe(include='all')

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

seconds 289.9940662384033


In [6]:
df_describe



,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
count,14300583,0,4079113,185928,13196,16138,3162,16333,12891,20305,...,999,33672,4292,2689,29968,204128,49937,1584,26878,5102
unique,14300583,0,616,9008,384,5710,500,2811,571,686,...,109,4446,875,1278,1226,5283,37670,14,1046,51
top,SAMN06189289,NaN,GRU,Unknown,Ustekinumab 45 mg,not collected,20,Missing: Not provided,missing,not applicable,...,0.3,not applicable,15.1 ÃÂµg/L,not applicable,0.25 g,9606,missing,Yes,proband,heterotroph
freq,1,NaN,1300152,20296,2907,428,418,2257,978,1547,...,146,2621,140,169,3927,92901,293,1053,4343,1057


In [7]:
df_describe.to_csv("harmonized-table.parquet_describe.tsv", sep="\t", header=True, index=True)

In [8]:
df_describe.iloc[3,].value_counts()

300      13
105      10
1         8
492       8
689       7
         ..
2388      1
56998     1
1367      1
344       1
621       1
Name: freq, Length: 347, dtype: int64

In [9]:
df_describe.loc[:, df_describe.isna().any()]

,urogenit_disord,douche,menarche,attribute,hrt
count,0,0,0,0,0
unique,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN


a first pass at values corresponding to 'no information', candidates for NA replacement for ML

In [10]:
missing_labels = ['NaN', 'Unknown', 'not collected', 'Missing: Not provided', 'missing', 'not applicable', 'not provided', '']

In [11]:
df['env_broad_scale'].describe()

count                      1545145
unique                       17586
top       human-associated habitat
freq                        116964
Name: env_broad_scale, dtype: object

In [12]:
df['env_broad_scale'].value_counts()

human-associated habitat      116964
missing                       108895
urban biome                    86064
not applicable                 72041
Bos taurus                     25803
                               ...  
decidious woodland                 1
Agricultural field 14              1
MIss                               1
Begonia elatior | stem rot         1
Mice colon                         1
Name: env_broad_scale, Length: 17586, dtype: int64

In [13]:
df['env_medium'].value_counts()

feces                           146111
faeces                          103521
soil                             99029
missing                          98177
not applicable                   71522
                                 ...  
CItr 15134                           1
agricultural waste material5         1
skin294                              1
Tank 1 Water 6                       1
sap from palm tree 1 time 35         1
Name: env_medium, Length: 22520, dtype: int64

In [14]:
!pip install -U sklearn


Requirement already up-to-date: sklearn in /Users/marcin/Documents/VIMSS/ontology/Sample/biosample-analysis/venv/lib/python3.7/site-packages (0.0)
Requirement not upgraded as not directly required: scikit-learn in /Users/marcin/Documents/VIMSS/ontology/Sample/biosample-analysis/venv/lib/python3.7/site-packages (from sklearn) (0.23.2)
Requirement not upgraded as not directly required: threadpoolctl>=2.0.0 in /Users/marcin/Documents/VIMSS/ontology/Sample/biosample-analysis/venv/lib/python3.7/site-packages (from scikit-learn->sklearn) (2.1.0)
Requirement not upgraded as not directly required: joblib>=0.11 in /Users/marcin/Documents/VIMSS/ontology/Sample/biosample-analysis/venv/lib/python3.7/site-packages (from scikit-learn->sklearn) (0.16.0)
Requirement not upgraded as not directly required: numpy>=1.13.3 in /Users/marcin/Documents/VIMSS/ontology/Sample/biosample-analysis/venv/lib/python3.7/site-packages (from scikit-learn->sklearn) (1.19.1)
Requirement not upgraded as not directly requir

In [15]:
laen = preprocessing.LabelEncoder()
df_encode = laen.fit_transform(df)

ValueError: y should be a 1d array, got an array of shape (14300584, 464) instead.

In [16]:
#onehotencoder = OneHotEncoder(categorical_features = [0])

#df_encode = OneHotEncoder().fit_transform(df)

In [18]:
#df_encode.to_parquet('harmonized-table__onehot.parquet')

In [6]:
#features = df.columns.drop('env_medium')
#type(features)

pandas.core.indexes.base.Index

In [17]:
df_human_index = df['taxonomy_id'].isin(['9606'])

In [18]:
df_human_index = list(compress(range(len(df_human_index )), df_human_index ))
df_human_index

[93,
 94,
 95,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 115,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 335,
 338,
 339,
 347,
 348,
 355,
 356,
 357,
 358,
 359,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 460,
 461,
 462

In [19]:
df_nonhuman = df.drop(df_human_index)
                 

In [20]:
df_nonhuman.shape

(7480877, 464)

In [21]:
df_nonhuman.to_parquet('harmonized-table__nonhuman.parquet')

In [22]:
df.env_package.unique()
env_packages = ['soil', 'Soil', 'sediment', 'plant-associated', 'water', 'seawater', 'wastewater', 'fresh water', 'freshwater sediment', 'sea water', 'sterile water', 'Coastal water body (ENVO:02000049)', 'sea water, [ENVO:00002149]', 'soil-associated', 'Sediment', 'biofilm', 'rock', 'plant', 'Water', 'marine sediment (ENVO:00002113)', 'Unknown', 'Sea water', 'cave', 'Deep-sea water', 'MIGS/MIMS/MIMARKS.sediment', 'MIGS/MIMS/MIMARKS.soil', 'MIGS/MIMS/MIMARKS.plant-associated', 'MIGS/MIMS/MIMARKS.water', 'MIGS/MIMS/MIMARKS.plant-associated', 'MIGS/MIMS/MIMARKS.air', 'ENVO:00000016']


In [23]:
df_env_index = df['env_package'].isin(env_packages)
df_env_index

0           False
1           False
2           False
3           False
4           False
            ...  
14300579    False
14300580    False
14300581    False
14300582    False
14300583    False
Name: env_package, Length: 14300584, dtype: bool

In [24]:
df_onlyenv = df[df_env_index]
df_onlyenv.shape

(44674, 464)

In [25]:
df_onlyenv.to_parquet('harmonized-table__envonly.parquet')

In [26]:
start_time = time.time()

df_onlyenv_describe = df_onlyenv.describe(include='all')

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

seconds 1.1152079105377197


In [27]:
df_onlyenv_describe

,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
count,44674,0,0,0,0,2877,992,5103,258,429,...,320,0,205,849,24,7847,1402,0,0,17
unique,44674,0,0,0,0,914,253,972,18,41,...,31,0,23,445,2,34,1202,0,0,5
top,SAMN02615886,NaN,NaN,NaN,NaN,Missing: Not Provided,6 m,Missing: Not provided,02/01/2011-12/01/2011,"Anti bird net applied, plants removed in bucke...",...,"0,981744518",NaN,15.1 ÃÂµg/L,"96,96",10 L,4577,BlockC3,NaN,NaN,photosynthetic
freq,1,NaN,NaN,NaN,NaN,276,284,2001,60,140,...,144,NaN,140,144,14,4739,24,NaN,NaN,9


In [28]:
df_onlyenv.head()

,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
19559,SAMEA1706329,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19560,SAMEA1706311,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19561,SAMEA1706320,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19562,SAMEA1706300,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19563,SAMEA1706316,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [29]:
#df_onlyenv_notNone = df_onlyenv[df_onlyenv != 'None'].index.values
#df_onlyenv_notNone.shape

#df_onlyenv.value_counts()

#df_onlyenv.groupby("None").count()

df_onlyenv.apply(pd.value_counts)

,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
"""This sample (TARA_A100000165) was collected during the Tara Oceans expedition (2009-2013) at station TARA_018 (latitudeN=35.759, longitudeE=14.2574) on date/time=2009-11-02T08:13, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (tbd-tbd micrometres), and stored in liquid nitrogen for later detection of prokaryote nucleic acid sequences by pyrosequencing methods, and for later metagenomics analysis. This sample has replicate sample(s): TARA_A100000166.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""This sample (TARA_A100000220) was collected during the Tara Oceans expedition (2009-2013) at station TARA_019 (latitudeN=34.2116, longitudeE=13.7684) on date/time=2009-11-11T16:24, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (0.22-1.6 micrometres), preserved on board with RNA-Later, and stored in liquid nitrogen for later detection of prokaryote nucleic acid sequences by pyrosequencing methods, and for later metagenomics/transcriptomics analysis.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""This sample (TARA_A100001026) was collected during the Tara Oceans expedition (2009-2013) at station TARA_032 (latitudeN=23.36, longitudeE=37.2183) on date/time=2010-01-11T07:21, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (0.22-1.6 micrometres), preserved on board with RNA-Later, and stored in liquid nitrogen for later detection of prokaryote nucleic acid sequences by pyrosequencing methods, and for later metagenomics/transcriptomics analysis. This sample has replicate sample(s): TARA_A100001027.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""This sample (TARA_A100001195) was collected during the Tara Oceans expedition (2009-2013) at station TARA_033 (latitudeN=21.9467, longitudeE=38.2517) on date/time=2010-01-13T07:16, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (0.22-1.6 micrometres), preserved on board with RNA-Later, and stored in liquid nitrogen for later detection of prokaryote nucleic acid sequences by pyrosequencing methods, and for later metagenomics/transcriptomics analysis. This sample has replicate sample(s): TARA_A100001197.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""This sample (TARA_A100001401) was collected during the Tara Oceans expedition (2009-2013) at station TARA_031 (latitudeN=27.16, longitudeE=27.1433) on date/time=2010-01-09T07:15, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (0.22-1.6 micrometres), preserved on

In [30]:
df_onlyenv_cp = df_onlyenv
cats = df_onlyenv_cp.columns
df_onlyenv_cp[cats] = df_onlyenv_cp[cats].astype("category")
df_onlyenv_cp[cats].describe()

/Users/marcin/Documents/VIMSS/ontology/Sample/biosample-analysis/venv/lib/python3.7/site-packages/pandas/core/frame.py:3062: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
count,44674,0,0,0,0,2877,992,5103,258,429,...,320,0,205,849,24,7847,1402,0,0,17
unique,44674,0,0,0,0,914,253,972,18,41,...,31,0,23,445,2,34,1202,0,0,5
top,SAMN14884473,NaN,NaN,NaN,NaN,Missing: Not Provided,6 m,Missing: Not provided,02/01/2011-12/01/2011,"Anti bird net applied, plants removed in bucke...",...,"0,981744518",NaN,15.1 ÃÂµg/L,"96,96",10 L,4577,BlockC3,NaN,NaN,photosynthetic
freq,1,NaN,NaN,NaN,NaN,276,284,2001,60,140,...,144,NaN,140,144,14,4739,24,NaN,NaN,9


In [32]:
df_onlyenv_value_counts = df_onlyenv.apply(pd.value_counts, axis=1)

In [33]:
df_onlyenv_describe.to_csv("harmonized-table.parquet_onlyenv_describe.tsv", sep="\t", header=True, index=True)

In [34]:
print(df_onlyenv_describe.loc['top', :].to_numpy())
type(df_onlyenv_describe.loc['top', :].to_numpy())

['SAMN02615886' nan nan nan nan 'Missing: Not Provided' '6 m'
 'Missing: Not provided' '02/01/2011-12/01/2011'
 'Anti bird net applied, plants removed in buckets before sampling because of difficulty in access'
 'lifeguard preservation solution' nan nan nan 'N/D' nan '67 kg N ha-1'
 nan nan nan nan '0.0' 'not applicable' '-1.9 degree Celsius' '0.5' nan
 'Immature' nan 'Healthy' 'daily' nan nan nan nan nan nan nan 'silt'
 'Missing: Not provided' nan nan 'Missing: Not provided' '8.9' nan
 'plants were grown in Duke Phytotron growth chambers to environmental variables were constant'
 nan nan '366.632' 'Missing: Not provided' nan nan nan nan nan
 'not applicable' nan nan nan nan 'Nipponbare' nan nan
 'ENVO:cropland biome' nan 'barley' '0.05 microgram per liter' '58%' nan
 nan 'multiple corer' nan nan 'ICMP:11032' nan '500mb' 'Not relevant' nan
 nan nan nan nan '21.1 oC (day); 15.6 oC (night)' 'Missing: Not Provided'
 nan 'Missing: Not provided' nan nan nan '0.50%' nan
 'Missing: Not provid

numpy.ndarray

In [35]:
#index = df_onlyenv_describe['top'].index[df_onlyenv_describe['top'].apply(np.isnan)]
df_onlyenv_index_notnan = np.where(df_onlyenv_describe.loc['top', :].notnull())[0]#np.where(df_onlyenv_describe.loc['top', :].isnan())
#index = list(np.where(df_onlyenv_describe.loc['top', :].isnan()[0]))

df_onlyenv_index_notnan

array([  0,   5,   6,   7,   8,   9,  10,  14,  16,  21,  22,  23,  24,
        26,  28,  29,  37,  38,  41,  42,  44,  47,  48,  54,  59,  62,
        64,  65,  66,  69,  72,  74,  75,  81,  82,  84,  88,  90,  91,
        92,  97, 101, 103, 104, 105, 106, 107, 108, 110, 115, 116, 120,
       123, 125, 126, 127, 128, 130, 132, 133, 136, 139, 140, 141, 145,
       146, 148, 154, 155, 156, 157, 159, 161, 162, 166, 167, 168, 169,
       170, 171, 173, 174, 177, 178, 181, 182, 183, 186, 187, 189, 199,
       200, 201, 202, 204, 205, 206, 207, 208, 210, 214, 215, 216, 217,
       222, 223, 224, 228, 232, 233, 237, 238, 239, 243, 244, 245, 247,
       254, 258, 263, 265, 270, 278, 281, 284, 285, 286, 287, 288, 289,
       291, 295, 296, 297, 298, 301, 302, 308, 310, 311, 313, 314, 315,
       316, 318, 320, 321, 322, 323, 324, 327, 328, 329, 331, 335, 336,
       338, 340, 342, 343, 345, 346, 347, 350, 356, 358, 361, 364, 366,
       367, 371, 373, 374, 375, 376, 381, 384, 385, 386, 387, 38

In [36]:
df_onlyenv_nona = df_onlyenv[df_onlyenv.columns[df_onlyenv_index_notnan]]

#gets opposite, so nans
#df_onlyenv_nona = df_onlyenv.drop(df_onlyenv.columns[df_onlyenv_index_notnan], axis=1)


In [37]:
df_onlyenv_nona_describe = df_onlyenv_nona.describe(include='all')
df_onlyenv_nona_describe

,accession,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,samp_mat_process,heavy_metals_meth,agrochem_addition,nitro,...,petroleum_hydrocarb,extreme_salinity,race,porosity,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,trophic_level
count,44674,2877,992,5103,258,429,5165,137,622,93,...,9,165,1,320,205,849,24,7847,1402,17
unique,44674,914,253,972,18,41,42,3,33,63,...,7,23,1,31,23,445,2,34,1202,5
top,SAMN14884473,Missing: Not Provided,6 m,Missing: Not provided,02/01/2011-12/01/2011,"Anti bird net applied, plants removed in bucke...",lifeguard preservation solution,N/D,67 kg N ha-1,0.0,...,iso alkanes,N/D,Unknown,"0,981744518",15.1 ÃÂµg/L,"96,96",10 L,4577,BlockC3,photosynthetic
freq,1,276,284,2001,60,140,3139,108,355,9,...,2,108,1,144,140,144,14,4739,24,9


In [38]:
df_onlyenv_nona.head()

,accession,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,samp_mat_process,heavy_metals_meth,agrochem_addition,nitro,...,petroleum_hydrocarb,extreme_salinity,race,porosity,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,trophic_level
19559,SAMEA1706329,NaN,NaN,NaN,NaN,NaN,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19560,SAMEA1706311,NaN,NaN,NaN,NaN,NaN,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19561,SAMEA1706320,NaN,NaN,NaN,NaN,NaN,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19562,SAMEA1706300,NaN,NaN,NaN,NaN,NaN,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19563,SAMEA1706316,NaN,NaN,NaN,NaN,NaN,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
env_series = df_onlyenv_nona['env_package']
print(type(env_series))
env_series

<class 'pandas.core.series.Series'>


19559       water
19560       water
19561       water
19562       water
19563       water
            ...  
14275237     soil
14275238     soil
14275239     soil
14275240     soil
14296019    Water
Name: env_package, Length: 44674, dtype: category
Categories (30, object): ['Coastal water body (ENVO:02000049)', 'Deep-sea water', 'ENVO:00000016', 'MIGS/MIMS/MIMARKS.air', ..., 'soil-associated', 'sterile water', 'wastewater', 'water']

In [40]:
df_onlyenv_nona_num_index = df_onlyenv_nona_describe.loc['top',:].str.isnumeric()
df_onlyenv_nona_num_index

accession              False
tot_nitro              False
tot_depth_water_col    False
nitrate                False
samp_store_dur         False
                       ...  
water_content          False
samp_vol_we_dna_ext    False
host_taxid              True
label                  False
trophic_level          False
Name: top, Length: 222, dtype: bool

In [41]:
df_onlyenv_nona_num = df_onlyenv_nona[df_onlyenv_nona.columns[df_onlyenv_nona_num_index]]



In [42]:
df_onlyenv_nona_num_describe = df_onlyenv_nona_num.describe(include='all')
df_onlyenv_nona_num_describe

,samp_salinity,altitude,taxonomy_id,fire,num_replicons,light_intensity,entrez_value,biosample_id,annual_season_precpt,host_taxid
count,76,6612,44674,291,392,65,4637,44674,1622,7847
unique,4,27,209,8,14,27,273,44674,132,34
top,8,0,410658,2004,1,5338,316748,998355,975,4577
freq,49,5915,8752,150,361,6,328,1,708,4739


In [43]:
df_onlyenv_nona_num.where(df_onlyenv_nona_num == '0 m').describe()

,samp_salinity,altitude,taxonomy_id,fire,num_replicons,light_intensity,entrez_value,biosample_id,annual_season_precpt,host_taxid
count,0,337,0,0,0,0,0,0,0,0
unique,0,1,0,0,0,0,0,0,0,0
top,NaN,0 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
#df_onlyenv_nona_num_str_row_index = df_onlyenv_nona_num.applymap(np.isreal).all(1)

In [93]:
#df_onlyenv_nona_num_str_row_index.describe()

count     34313
unique        1
top       False
freq      34313
dtype: object

In [89]:
#df_onlyenv_nona_num_nostr = df_onlyenv_nona_num[df_onlyenv_nona_num_str_row_index]

In [45]:
df_onlyenv_nona_num.head()

,samp_salinity,altitude,taxonomy_id,fire,num_replicons,light_intensity,entrez_value,biosample_id,annual_season_precpt,host_taxid
19559,NaN,NaN,449393,NaN,NaN,NaN,NaN,26206,NaN,NaN
19560,NaN,NaN,449393,NaN,NaN,NaN,NaN,26207,NaN,NaN
19561,NaN,NaN,449393,NaN,NaN,NaN,NaN,26208,NaN,NaN
19562,NaN,NaN,449393,NaN,NaN,NaN,NaN,26209,NaN,NaN
19563,NaN,NaN,449393,NaN,NaN,NaN,NaN,26210,NaN,NaN


In [46]:
#df_onlyenv_nona_num_nostr = pd.to_numeric(df_onlyenv_nona_num_nostr)
df_onlyenv_nona_num_nostr = df_onlyenv_nona_num.apply(pd.to_numeric, errors="coerce")
#df_onlyenv_nona_num_nostr = df_onlyenv_nona_num.astype(int)

In [47]:
df_onlyenv_nona_num_nostr.head()

,samp_salinity,altitude,taxonomy_id,fire,num_replicons,light_intensity,entrez_value,biosample_id,annual_season_precpt,host_taxid
19559,NaN,NaN,449393,NaN,NaN,NaN,NaN,26206,NaN,NaN
19560,NaN,NaN,449393,NaN,NaN,NaN,NaN,26207,NaN,NaN
19561,NaN,NaN,449393,NaN,NaN,NaN,NaN,26208,NaN,NaN
19562,NaN,NaN,449393,NaN,NaN,NaN,NaN,26209,NaN,NaN
19563,NaN,NaN,449393,NaN,NaN,NaN,NaN,26210,NaN,NaN


In [48]:
print(df_onlyenv_nona_num_nostr.shape)
df_onlyenv_nona_num_nostr.describe(include='all')

(44674, 10)


,samp_salinity,altitude,taxonomy_id,fire,num_replicons,light_intensity,entrez_value,biosample_id,annual_season_precpt,host_taxid
count,76.000000,5972.000000,4.467400e+04,150.0,383.000000,52.000000,4.637000e+03,4.467400e+04,1255.000000,7.698000e+03
mean,7.726447,38.528801,6.123975e+05,2004.0,2873.796345,26950.969231,8.175037e+05,7.192416e+06,984.823363,1.687614e+05
std,0.381711,424.608695,3.914641e+05,0.0,51339.056773,21088.525193,3.709119e+06,4.016092e+06,557.667035,3.789085e+05
min,7.100000,0.000000,2.000000e+00,2004.0,1.000000,4900.000000,4.540000e+02,2.620600e+04,3.160000,3.208000e+03
25%,7.440000,0.000000,4.106580e+05,2004.0,1.000000,12300.000000,1.892730e+05,4.478837e+06,870.000000,4.577000e+03
50%,8.000000,0.000000,4.127550e+05,2004.0,1.000000,23400.000000,2.150120e+05,6.058498e+06,975.000000,4.577000e+03
75%,8.000000,0.000000,9.399280e+05,2004.0,1.000000,30000.000000,2.327220e+05,9.109101e+06,975.000000,4.577000e+03
max,8.000000,5450.000000,2.742685e+06,2004.0,1000000.000000,85422.800000,2.468230e+07,1.576924e+07,6200.000000,1.671699e+06


In [49]:
df_onlyenv_nona_num_nostr.isnull()

,samp_salinity,altitude,taxonomy_id,fire,num_replicons,light_intensity,entrez_value,biosample_id,annual_season_precpt,host_taxid
19559,True,True,False,True,True,True,True,False,True,True
19560,True,True,False,True,True,True,True,False,True,True
19561,True,True,False,True,True,True,True,False,True,True
19562,True,True,False,True,True,True,True,False,True,True
19563,True,True,False,True,True,True,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...
14275237,True,True,False,True,True,True,True,False,True,True
14275238,True,True,False,True,True,True,True,False,True,True
14275239,True,True,False,True,True,True,True,False,True,True
14275240,True,True,False,True,True,True,True,False,True,True


In [50]:
df_onlyenv_nona_num_nostr_nan_index = df_onlyenv_nona_num_nostr.isnull()
df_onlyenv_nona_num_nostr_nan_rows_index = df_onlyenv_nona_num_nostr_nan_index.any(axis=1)
print(df_onlyenv_nona_num_nostr_nan_rows_index)
df_onlyenv_nona_num_nostr_nan_rows_index_rows = df_onlyenv_nona_num_nostr[df_onlyenv_nona_num_nostr_nan_rows_index]
df_onlyenv_nona_num_nostr_nan_rows_index_rows

19559       True
19560       True
19561       True
19562       True
19563       True
            ... 
14275237    True
14275238    True
14275239    True
14275240    True
14296019    True
Length: 44674, dtype: bool


,samp_salinity,altitude,taxonomy_id,fire,num_replicons,light_intensity,entrez_value,biosample_id,annual_season_precpt,host_taxid
19559,NaN,NaN,449393,NaN,NaN,NaN,NaN,26206,NaN,NaN
19560,NaN,NaN,449393,NaN,NaN,NaN,NaN,26207,NaN,NaN
19561,NaN,NaN,449393,NaN,NaN,NaN,NaN,26208,NaN,NaN
19562,NaN,NaN,449393,NaN,NaN,NaN,NaN,26209,NaN,NaN
19563,NaN,NaN,449393,NaN,NaN,NaN,NaN,26210,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
14275237,NaN,NaN,256318,NaN,NaN,NaN,NaN,15737080,NaN,NaN
14275238,NaN,NaN,256318,NaN,NaN,NaN,NaN,15737081,NaN,NaN
14275239,NaN,NaN,256318,NaN,NaN,NaN,NaN,15737082,NaN,NaN
14275240,NaN,NaN,256318,NaN,NaN,NaN,NaN,15737083,NaN,NaN


In [51]:
df_onlyenv_nona_num_nostr_nans = df_onlyenv_nona_num_nostr.isnull().sum(axis=1)
df_onlyenv_nona_num_nostr_nans.describe()

count    44674.000000
mean         7.547321
std          0.608450
min          5.000000
25%          7.000000
50%          8.000000
75%          8.000000
max          8.000000
dtype: float64

In [53]:
df_onlyenv_nona_num_nostr.shape

(44674, 10)

In [55]:
df_onlyenv_nona_num_nostr.to_csv("harmonized-table_onlyenv_nona_num.tsv", sep="\t", header=True, index=True)

In [52]:
start_time = time.time()

features = []

y= env_series#df['env_medium']
X= df_onlyenv_nona_num_nostr #df[features]

###TODO: increase splits
dtree=DecisionTreeClassifier(min_samples_split=20, random_state=9999)
#df_trim = df.drop('env_medium', axis=1) 
dtree.fit(X,y)
print('Decision Tree Classifer Created')

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [ ]:
visualize_tree(dt, features)

In [ ]:
start_time = time.time()

dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data, feature_names=labels,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))